# Create html page with ML questions 

In [1]:
from yattag import Doc
import json
import bs4
import os

In [2]:
with open('../data/ml_questions_tmp.json') as f:
    data = json.load(f)

In [3]:
doc, tag, text, line = Doc().ttl()
images = os.listdir('../copy_files/ml/images')
title = 'Machine Learning questions'
num_questions = 0

def get_content_type(content):
    content_type = None

    if content in images:
        content_type = 'img'
    elif type(content) is list:
        content_type = 'list'
    else:
        content_type = 'str'

    return content_type

def get_content(content_list):
    last_type = 'init'
    for content in content_list:
        content_type = get_content_type(content)

        if content_type == 'img':
            if last_type == 'str':
                line('p', '')
            doc.stag('img', src=f'images/{content}')
            if content != content_list[-1]:
                line('p', '')
        elif content_type == 'list':
            with tag('ul'):
                for item in content:
                    line('li', item)
        elif content_type == 'str':
            if last_type == 'str':
                text('<br>')
            text(content)
        else:
            print('Content type not defined')

        last_type = content_type


def page_content():
    line('devsite-mathjax', '', config='TeX-AMS-MML_SVG')
    with tag('section', id='home_section', klass="expandable", align="right"):
        line('b', '', klass="showalways")
        with tag('div', align="left"):
            line('h1', title)
            text('<br><br>')
            url = 'https://eavelardev.github.io'
            line('button', 'My AI Portfolio', onclick=f"location.href='{url}';")
            line('button', 'Shuffle questions', onclick="shuffleQuestions()")
            line('button', 'Reset', onclick="resetQuestions()")
            with tag('label', klass="toggle"):
                f = 'showSubQuestions()'
                doc.stag('input', klass='toggle-checkbox', type='checkbox', onclick=f)
                line('div', '', klass='toggle-switch')
                line('span', '', klass='toggle-label')
            text('<br><br>')
            with tag('div', klass='checkboxes'):
                f = 'filterSelection'
                doc.stag('input', id="official", type='checkbox', onclick=f"{f}('official')")
                line('label', 'Official', kfor='official')
                doc.stag('input', id="gdgs", type='checkbox', onclick=f"{f}('gdgs')")
                line('label', 'GDGs', kfor='gdgs')
                doc.stag('input', id="ml_path", type='checkbox', onclick=f"{f}('ml_path')")
                line('label', 'ML path', kfor='ml_path')
                doc.stag('input', id="google_ml", type='checkbox', onclick=f"{f}('google_ml')")
                line('label', 'Google ML', kfor='google_ml')
                doc.stag('input', id="whizlabs", type='checkbox', onclick=f"{f}('whizlabs')")
                line('label', 'Whizlabs', kfor='whizlabs')
                doc.stag('input', id="udemy", type='checkbox', onclick=f"{f}('udemy')")
                line('label', 'Udemy', kfor='udemy')
                doc.stag('input', id="books", type='checkbox', onclick=f"{f}('books')")
                line('label', 'Books', kfor='books')
                with tag('div'):
                    doc.stag('input', id="incorrects", type='checkbox', onclick=f"{f}('incorrects')")
                    line('label', 'Only incorrects', kfor='incorrects')
            text('<br>')
            doc.stag('input', id='myRange', klass='slider', type='range', min='1')
    with tag('div', klass='devsite-steps'):
        with tag('div', klass='steps-previous steps-direction'):
            with tag('a', id='link-arrow-left'):
                line('span', 'arrow_back', 
                     klass='steps-link-arrow steps-link-arrow-left material-icons')
        with tag('div', klass='steps-next steps-direction'):
            with tag('a', id='link-arrow-right'):
                line('span', 'arrow_forward', 
                     klass='steps-link-arrow steps-link-arrow-right material-icons')
    with tag('div', klass='questions'):
        for qtag, sources in data.items():
            questions = sum(sources.values(), [])
            global num_questions
            num_questions += len(questions)
            for q in questions:
                with tag('div', klass=f'filterDiv {qtag}'):
                    get_content(q['intro'])
                    with tag('div'):
                        with tag('devsite-multiple-choice'):
                            with tag('div'):
                                    with tag('div'):
                                        get_content(q['body'])
                            for val in q['options']:
                                answer = 'correct' if val['answer'] else ''
                                with tag('div', answer):
                                    with tag('div'):
                                        get_content(val['option'])
                                    with tag('div'):
                                        get_content(val['explanation'])
                    if q['feedback'] or q['references']:    
                        with tag('section', klass='expandable', 
                                align='right'):
                            line('b', '', klass='showalways')
                            line('p', '')
                            with tag('div', klass='expand-background', 
                                    align='left'):
                                get_content(q['feedback'])
                                if q['references']:
                                    if q['feedback']:
                                        cont = q['feedback'][-1]
                                        if get_content_type(cont) == 'str':
                                            text('<br><br>')
                                        elif get_content_type(cont) == 'img':
                                            text('<br>')
                                    line('b', 'References:')
                                    with tag('ul'):
                                        for cont in q['references']:
                                            with tag('li'):
                                                if cont.startswith('http'):
                                                    line('a', cont, 
                                                        href=cont, 
                                                        target='_blank')
                                                else:
                                                    text(cont)
                    text('<br>')


doc.asis('<!DOCTYPE html>')
with tag('html'):
    with tag('head'):
        line('title', title)
        doc.stag('link', rel='stylesheet', href='css/css.css')
        doc.stag('link', rel='stylesheet', href='css/css2.css')
        doc.stag('link', rel='stylesheet', href='css/myapp.css')
        doc.stag('link', rel='stylesheet', href='css/cc.css')
        doc.stag('link', rel='shortcut icon', href='../_static/logo.png')
    with tag('body', layout='docs'):
        with tag('main', klass='devsite-main-content'):
            with tag('devsite-content'):
                with tag('article', klass='devsite-article'):
                    page_content()
        line('script', '', src='js/questions_loader.js', type='text/javascript')

formatter = bs4.formatter.HTMLFormatter(indent=2)
html = bs4.BeautifulSoup(doc.getvalue()).prettify(formatter=formatter)

with open('../copy_files/ml/questions.html', 'w') as f:
    f.write(html)

print(f'Num questions: {num_questions}')

Num questions: 522
